In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/embedding/fine_tuned_embedding_B.npy
/kaggle/input/embedding/fine_tuned_embedding_p.npy
/kaggle/input/embedding/fine_tuned_embedding_A.npy
/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [23]:
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses
# GPUが利用可能か確認
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cuda


In [25]:
pip install textstat nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 24.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [26]:
# テキスト分析・自然言語処理
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
# 可読性スコア計算
import textstat

# 文章類似度計算
from sklearn.metrics.pairwise import cosine_similarity

# 正規表現
import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# データの読み込み
train_data = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/train.csv')
test_data = pd.read_csv(r'/kaggle/input/llm-classification-finetuning/test.csv')
labels = np.zeros(len(train_data), dtype=int)
labels[train_data['winner_model_a'] == 1] = 0  # Aの勝ち
labels[train_data['winner_model_b'] == 1] = 1  # Bの勝ち
labels[train_data['winner_tie'] == 1] = 2    # 同点
fine_tuned_embedding_p = np.load("/kaggle/input/embedding/fine_tuned_embedding_p.npy")
fine_tuned_embedding_A = np.load("/kaggle/input/embedding/fine_tuned_embedding_A.npy")
fine_tuned_embedding_B = np.load("/kaggle/input/embedding/fine_tuned_embedding_B.npy")


In [4]:
# def extract_better_worse_responses(train_data):
#     """
#     トレーニングデータから好まれる回答と好まれない回答のペアを抽出する関数
    
#     Args:
#         train_data (pd.DataFrame): 元のトレーニングデータフレーム
        
#     Returns:
#         tuple: (prompts, better_responses, worse_responses)のタプル
#     """
#     prompts = []
#     better_responses = []
#     worse_responses = []
    
#     for _, row in train_data.iterrows():
#         prompt = row['prompt']
        
#         # 好まれる回答と好まれない回答を特定
#         if row['winner_model_a'] == 1:  # モデルAが勝者
#             better_response = row['response_a']
#             worse_response = row['response_b']
#             prompts.append(prompt)
#             better_responses.append(better_response)
#             worse_responses.append(worse_response)
            
#         elif row['winner_model_b'] == 1:  # モデルBが勝者
#             better_response = row['response_b']
#             worse_response = row['response_a']
#             prompts.append(prompt)
#             better_responses.append(better_response)
#             worse_responses.append(worse_response)
            
#         # 同点の場合はスキップ（または別の処理を追加することも可能）
#         # 同点の場合も含める場合は以下のコメントを解除
#         #elif row['winner_tie'] == 1:
#             # 同点の場合の処理（例えば、両方とも「better」として扱うなど）
    
#     return prompts, better_responses, worse_responses

# # 関数を使用してデータを抽出
# prompts, better_responses, worse_responses = extract_better_worse_responses(train_data)

# print(f"抽出されたサンプル数: {len(prompts)}")
# print(f"最初の例:\nプロンプト: {prompts[0][:100]}...\n好まれる回答: {better_responses[0][:100]}...\n好まれない回答: {worse_responses[0][:100]}...")

In [5]:
# # Wandbをオフにするため、環境変数を設定
# import os
# os.environ["WANDB_DISABLED"] = "true"
# # モデルのロード
# model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# # 学習率の設定（通常はデフォルトより小さい値で開始）
# learning_rate = 2e-5  # SBERTの微調整には2e-5〜5e-5が一般的

# # ウェイトディケイの設定
# weight_decay = 0.01  # 過学習防止のため

# # マージン値の設定（トリプレットロスのパラメータ）
# margin = 0.5  # 正例と負例の埋め込み間の距離差をどれだけ取るか

# # トレーニングサンプルの作成
# train_examples = []
# for prompt, better, worse in zip(prompts, better_responses, worse_responses):
#     train_examples.append(InputExample(texts=[prompt, better, worse]))

# # バッチサイズの設定（大きすぎるとメモリ不足に、小さすぎると学習が安定しない）
# batch_size = 16

# # エポック数（多すぎると過学習するリスクあり）
# num_epochs = 3

# # データローダーの作成
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# # トリプレットロスの設定（マージンを指定）
# train_loss = losses.TripletLoss(model=model, triplet_margin=margin)

# # 早期停止のコールバック設定
# from sentence_transformers.evaluation import TripletEvaluator
# # 検証用データセットを作成（全データの一部を使用）
# dev_samples = train_examples[:len(train_examples)//10]  # 10%を検証に使用
# evaluator = TripletEvaluator.from_input_examples(dev_samples)

# # オプティマイザの設定
# optimizer_params = {'lr': learning_rate, 'weight_decay': weight_decay}

# # ファインチューニングの実行
# model.fit(
#     train_objectives=[(train_dataloader, train_loss)],
#     epochs=num_epochs,
#     evaluator=evaluator,
#     evaluation_steps=1000,  # 1000バッチごとに評価
#     warmup_steps=int(len(train_dataloader) * 0.1),  # 全体の10%をウォームアップに
#     optimizer_params=optimizer_params,
#     optimizer_class=torch.optim.AdamW,  # AdamWオプティマイザを使用
#     scheduler='WarmupLinear',  # 学習率スケジューラ
#     show_progress_bar=True,
#     output_path='fine-tuned-sbert-model',
#     save_best_model=True  # 最良のモデルを保存
# )
# # ファインチューニングされたモデルを使用してデータを再エンコード
# fine_tuned_embedding_p = model.encode(train_data["prompt"])
# fine_tuned_embedding_A = model.encode(train_data["response_a"])
# fine_tuned_embedding_B = model.encode(train_data["response_b"])

In [6]:
# np.save('fine_tuned_embedding_p.npy', fine_tuned_embedding_p)
# np.save('fine_tuned_embedding_A.npy', fine_tuned_embedding_A)
# np.save('fine_tuned_embedding_B.npy', fine_tuned_embedding_B)

In [ ]:
# 前処理段階
def extract_and_save_features(data_csv, output_path):
    # 追加の特徴を計算
    linguistic_features_a = np.array([extract_linguistic_features(text) for text in train_data["response_a"]])
    linguistic_features_b = np.array([extract_linguistic_features(text) for text in train_data["response_b"]])
    
    readability_scores_a = np.array([compute_readability_scores(text) for text in train_data["response_a"]])
    readability_scores_b = np.array([compute_readability_scores(text) for text in train_data["response_b"]])
    
    # プロンプトと応答の関係性特徴
    relation_features_a = np.array([compute_prompt_response_relation(p, r) for p, r in zip(train_data["prompt"], train_data["response_a"])])
    relation_features_b = np.array([compute_prompt_response_relation(p, r) for p, r in zip(train_data["prompt"], train_data["response_b"])])
    
    # セマンティック一貫性特徴
    semantic_features_a = np.array([semantic_coherence_features(p, r) for p, r in zip(train_data["prompt"], train_data["response_a"])])
    semantic_features_b = np.array([semantic_coherence_features(p, r) for p, r in zip(train_data["prompt"], train_data["response_b"])])

    np.save("linguistic_features_a.npy", linguistic_features_a)
    np.save("linguistic_features_b.npy", linguistic_features_b)
    np.save("readability_scores_a.npy", readability_scores_a)
    np.save("readability_scores_b.npy", readability_scores_b)
    np.save("relation_features_a.npy", relation_features_a)
    np.save("relation_features_b.npy", relation_features_b)
    np.save("semantic_features_a.npy", semantic_features_a)
    np.save("semantic_features_b.npy", semantic_features_b)

In [ ]:
def load_features(feature_dir='.'):
    """
    保存された特徴ファイルを読み込む関数
    
    Args:
        feature_dir (str): 特徴ファイルが保存されているディレクトリパス
        
    Returns:
        dict: 読み込まれた特徴を含む辞書
    """
    features = {}
    
    # 埋め込み特徴の読み込み
    features['prompt_embeddings'] = np.load(f"{feature_dir}/fine_tuned_embedding_p.npy")
    features['response_a_embeddings'] = np.load(f"{feature_dir}/fine_tuned_embedding_A.npy")
    features['response_b_embeddings'] = np.load(f"{feature_dir}/fine_tuned_embedding_B.npy")
    
    # 言語的特徴の読み込み
    features['linguistic_features_a'] = np.load(f"{feature_dir}/linguistic_features_a.npy")
    features['linguistic_features_b'] = np.load(f"{feature_dir}/linguistic_features_b.npy")
    
    # 可読性スコアの読み込み
    features['readability_scores_a'] = np.load(f"{feature_dir}/readability_scores_a.npy")
    features['readability_scores_b'] = np.load(f"{feature_dir}/readability_scores_b.npy")
    
    # 関係性特徴の読み込み
    features['relation_features_a'] = np.load(f"{feature_dir}/relation_features_a.npy")
    features['relation_features_b'] = np.load(f"{feature_dir}/relation_features_b.npy")
    
    # セマンティック特徴の読み込み
    features['semantic_features_a'] = np.load(f"{feature_dir}/semantic_features_a.npy")
    features['semantic_features_b'] = np.load(f"{feature_dir}/semantic_features_b.npy")
    
    print(f"特徴が正常に読み込まれました。データサイズ: {len(features['prompt_embeddings'])}")
    
    return features

In [29]:
class MultiModalDataset(Dataset):
    """
    マルチモーダル特徴を持つデータセットクラス
    """
    def __init__(self, prompt_embeddings, response_a_embeddings, response_b_embeddings, 
                 linguistic_features_a, linguistic_features_b,
                 readability_a, readability_b,
                 relation_a, relation_b,
                 semantic_a, semantic_b,
                 labels=None):
        """
        初期化関数
        
        Args:
            prompt_embeddings: プロンプトの埋め込み
            response_a_embeddings: 回答Aの埋め込み
            response_b_embeddings: 回答Bの埋め込み
            linguistic_features_a: 回答Aの言語的特徴
            linguistic_features_b: 回答Bの言語的特徴
            readability_a: 回答Aの可読性スコア
            readability_b: 回答Bの可読性スコア
            relation_a: プロンプトと回答Aの関係特徴
            relation_b: プロンプトと回答Bの関係特徴
            semantic_a: 回答Aの意味的特徴
            semantic_b: 回答Bの意味的特徴
            labels: ラベル（0: Aが好まれる, 1: Bが好まれる, 2: 同等）
        """
        self.prompt_embeddings = torch.tensor(prompt_embeddings, dtype=torch.float32)
        self.response_a_embeddings = torch.tensor(response_a_embeddings, dtype=torch.float32)
        self.response_b_embeddings = torch.tensor(response_b_embeddings, dtype=torch.float32)
        
        self.linguistic_features_a = torch.tensor(linguistic_features_a, dtype=torch.float32)
        self.linguistic_features_b = torch.tensor(linguistic_features_b, dtype=torch.float32)
        
        self.readability_a = torch.tensor(readability_a, dtype=torch.float32)
        self.readability_b = torch.tensor(readability_b, dtype=torch.float32)
        
        self.relation_a = torch.tensor(relation_a, dtype=torch.float32)
        self.relation_b = torch.tensor(relation_b, dtype=torch.float32)
        
        self.semantic_a = torch.tensor(semantic_a, dtype=torch.float32)
        self.semantic_b = torch.tensor(semantic_b, dtype=torch.float32)
        
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.long)
        else:
            self.labels = None
            
        self.has_labels = labels is not None
        
    def __len__(self):
        """データセットの長さを返す"""
        return len(self.prompt_embeddings)
    
    def __getitem__(self, idx):
        """インデックスに対応するデータを返す"""
        if self.has_labels:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx],
                self.linguistic_features_a[idx],
                self.linguistic_features_b[idx],
                self.readability_a[idx],
                self.readability_b[idx],
                self.relation_a[idx],
                self.relation_b[idx],
                self.semantic_a[idx],
                self.semantic_b[idx],
                self.labels[idx]
            )
        else:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx],
                self.linguistic_features_a[idx],
                self.linguistic_features_b[idx],
                self.readability_a[idx],
                self.readability_b[idx],
                self.relation_a[idx],
                self.relation_b[idx],
                self.semantic_a[idx],
                self.semantic_b[idx]
            )

In [27]:
def extract_linguistic_features(text):
    features = {}
    # 長さ関連の特徴
    features['char_count'] = len(text)
    features['word_count'] = len(text.split())
    features['sent_count'] = len(re.split(r'[.!?]+', text))
    features['avg_word_length'] = sum(len(word) for word in text.split()) / max(1, len(text.split()))
    features['avg_sent_length'] = features['word_count'] / max(1, features['sent_count'])
    
    # 文体関連の特徴
    features['question_marks'] = text.count('?') / max(1, features['sent_count'])
    features['exclamation_marks'] = text.count('!') / max(1, features['sent_count'])
    features['capital_ratio'] = sum(1 for c in text if c.isupper()) / max(1, len(text))
    
    # 語彙の複雑さ
    words = text.lower().split()
    unique_words = set(words)
    features['lexical_diversity'] = len(unique_words) / max(1, len(words))
    
    # 特定語彙の使用頻度
    hedging_words = ['maybe', 'perhaps', 'possibly', 'might', 'could', 'seem', 'appear']
    features['hedging_ratio'] = sum(words.count(word) for word in hedging_words) / max(1, len(words))
    
    # 主観性/客観性指標
    subj_words = ['I', 'my', 'mine', 'me', 'personally', 'believe', 'think', 'feel']
    features['subjectivity'] = sum(words.count(word.lower()) for word in subj_words) / max(1, len(words))
    
    return features

def compute_readability_scores(text):
    scores = {}
    # Flesch Reading Ease
    scores['flesch_reading_ease'] = textstat.flesch_reading_ease(text)
    # Flesch-Kincaid Grade Level
    scores['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)
    # SMOG Index
    scores['smog_index'] = textstat.smog_index(text)
    return scores

def compute_prompt_response_relation(prompt, response):
    relation_features = {}
    # プロンプトの質問に対する直接答えの指標
    prompt_lower = prompt.lower()
    if '?' in prompt:
        question_words = ['what', 'how', 'why', 'when', 'where', 'who', 'which']
        detected_q_words = [w for w in question_words if w in prompt_lower]
        
        # 質問語に対する回答の始まり方を分析
        response_start = response.split()[:10]
        relation_features['direct_answer'] = any(
            response.lower().startswith(starter) 
            for starter in ["yes", "no", "the", "it is", "there are"]
        )
    
    # 内容の一致度
    prompt_tokens = set(prompt_lower.split())
    response_tokens = set(response.lower().split())
    relation_features['token_overlap'] = len(prompt_tokens.intersection(response_tokens)) / max(1, len(prompt_tokens))
    
    # プロンプトの長さと応答の長さの比率
    relation_features['length_ratio'] = len(response) / max(1, len(prompt))
    
    return relation_features


def semantic_coherence_features(prompt, response):
    prompt_emb = model.encode(prompt)
    
    # 応答を文ごとに分割して埋め込む
    sentences = re.split(r'[.!?]+', response)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    if len(sentences) <= 1:
        return {
            'sent_coherence': 1.0,
            'prompt_relevance': cosine_similarity([prompt_emb], [model.encode(response)])[0][0]
        }
    
    # 文間の一貫性 (隣接する文のコサイン類似度の平均)
    sent_embeddings = model.encode(sentences)
    coherence_scores = []
    
    for i in range(len(sent_embeddings) - 1):
        sim = cosine_similarity([sent_embeddings[i]], [sent_embeddings[i+1]])[0][0]
        coherence_scores.append(sim)
    
    # プロンプトとの関連性 (最初と最後の文がプロンプトにどれだけ関連しているか)
    prompt_first_sim = cosine_similarity([prompt_emb], [sent_embeddings[0]])[0][0]
    prompt_last_sim = cosine_similarity([prompt_emb], [sent_embeddings[-1]])[0][0]
    
    return {
        'sent_coherence': sum(coherence_scores) / max(1, len(coherence_scores)),
        'prompt_first_sent': prompt_first_sim,
        'prompt_last_sent': prompt_last_sim,
        'prompt_relevance': cosine_similarity([prompt_emb], [model.encode(response)])[0][0]
    }



In [28]:
class MultiModalComparisonModel(nn.Module):
    def __init__(self, emb_dim=384, ling_feat_dim=20, readability_dim=3, relation_dim=3, semantic_dim=4):
        super(MultiModalComparisonModel, self).__init__()
        
        # 各モダリティの特徴抽出
        self.embedding_encoder = nn.Sequential(
            nn.Linear(emb_dim * 2, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128)
        )
        
        self.linguistic_encoder = nn.Sequential(
            nn.Linear(ling_feat_dim * 2, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 32)
        )
        
        self.readability_encoder = nn.Sequential(
            nn.Linear(readability_dim * 2, 32),
            nn.LayerNorm(32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        
        self.relation_encoder = nn.Sequential(
            nn.Linear(relation_dim * 2, 32),
            nn.LayerNorm(32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        
        self.semantic_encoder = nn.Sequential(
            nn.Linear(semantic_dim * 2, 32),
            nn.LayerNorm(32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        
        # 注意機構を用いたモダリティ間の重み付け
        self.modality_attention = nn.MultiheadAttention(
            embed_dim=16, 
            num_heads=2, 
            batch_first=True
        )
        
        # モダリティの融合
        combined_dim = 128 + 32 + 16 + 16 + 16  # すべてのエンコーダー出力の合計
        
        self.fusion_layer = nn.Sequential(
            nn.Linear(combined_dim, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # 差分層（A-Bの特徴差分を計算）
        self.diff_layer = nn.Sequential(
            nn.Linear(64 * 2, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # 最終分類層
        self.classifier = nn.Linear(64, 3)  # 3クラス分類
        
    def encode_response(self, prompt_emb, resp_emb, ling_feat, readability, relation, semantic):
        # 各モダリティの特徴抽出
        emb_feat = self.embedding_encoder(torch.cat([prompt_emb, resp_emb], dim=1))
        ling_feat = self.linguistic_encoder(ling_feat)
        read_feat = self.readability_encoder(readability)
        rel_feat = self.relation_encoder(relation)
        sem_feat = self.semantic_encoder(semantic)
        
        # モダリティの結合と注意機構による重み付け
        # 各モダリティの特徴を16次元に揃える
        modal_feats = torch.stack([
            F.pad(ling_feat, (0, 16-ling_feat.size(1))),
            F.pad(read_feat, (0, 16-read_feat.size(1))),
            F.pad(rel_feat, (0, 16-rel_feat.size(1))),
            F.pad(sem_feat, (0, 16-sem_feat.size(1)))
        ], dim=1)  # [batch_size, 4, 16]
        
        # モダリティ間の相互作用を計算
        attn_out, _ = self.modality_attention(modal_feats, modal_feats, modal_feats)
        
        # 注意機構の出力を平坦化 [batch_size, 4*16]
        attn_out = attn_out.reshape(attn_out.size(0), -1)
        
        # 埋め込み特徴と注意機構出力を結合
        combined = torch.cat([emb_feat, attn_out], dim=1)
        
        # 融合
        fused = self.fusion_layer(combined)
        
        return fused
        
    def forward(self, prompt_emb, resp_a_emb, resp_b_emb, 
               ling_feat_a, ling_feat_b, 
               read_a, read_b, 
               rel_a, rel_b, 
               sem_a, sem_b):
        
        # 各応答の特徴を抽出
        resp_a_feat = self.encode_response(prompt_emb, resp_a_emb, ling_feat_a, read_a, rel_a, sem_a)
        resp_b_feat = self.encode_response(prompt_emb, resp_b_emb, ling_feat_b, read_b, rel_b, sem_b)
        
        # 差分特徴（両方向の差分を考慮）
        diff_a_b = resp_a_feat - resp_b_feat
        diff_b_a = resp_b_feat - resp_a_feat
        
        # 差分の絶対値や乗算などの関係性特徴
        diff_abs = torch.abs(diff_a_b)
        diff_mul = resp_a_feat * resp_b_feat
        
        # すべての差分特徴を結合
        diff_combined = torch.cat([diff_a_b, diff_abs], dim=1)
        
        # 差分特徴から最終分類
        diff_feat = self.diff_layer(diff_combined)
        logits = self.classifier(diff_feat)
        
        return logits

In [7]:

class ImprovedResponseEvaluator(nn.Module):
    """
    プロンプトを基準に回答を評価する改善版モデル
    
    シンプルで効率的なアーキテクチャを採用し、
    クロスアテンション、残差接続、明示的な差分表現により
    回答の好まれやすさを予測します。
    """
    def __init__(self, input_dim=384, hidden_dim=256, num_heads=6, dropout=0.15):
        """
        初期化関数
        
        Args:
            input_dim (int): 入力特徴量の次元数（SBERTの場合通常は384次元）
            hidden_dim (int): 隠れ層の次元数
            num_heads (int): 注意機構のヘッド数
            dropout (float): ドロップアウト率
        """
        super(ImprovedResponseEvaluator, self).__init__()
        
        # 入出力の次元
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # クロスアテンション（プロンプトと回答間の関係を処理）
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=input_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # 特徴抽出器（シンプル化）
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.ReLU()
        )
        
        # 比較層（差分情報を含む）
        self.comparison_layer = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),  # 3倍の次元（A特徴、B特徴、差分）
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.ReLU()
        )
        
        # 出力層（3クラス分類）
        self.classifier = nn.Linear(hidden_dim // 2, 3)
        
        # 補助的な評価モジュール（各回答の個別スコア計算用）
        self.response_scorer = nn.Sequential(
            nn.Linear(hidden_dim, 1)
        )
        
        # モデルの重みを初期化
        self._init_weights()
        
    def _init_weights(self):
        """モデルの重みを適切に初期化する関数"""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def process_response(self, prompt_encoding, response_encoding):
        """
        プロンプトと1つの回答の関係を処理する
        
        Args:
            prompt_encoding (Tensor): プロンプトのエンコーディング [batch_size, 1, input_dim]
            response_encoding (Tensor): 回答のエンコーディング [batch_size, 1, input_dim]
            
        Returns:
            Tensor: 回答の特徴表現 [batch_size, hidden_dim]
        """
        # クロスアテンション（プロンプトをクエリ、回答をキー・バリューとして）
        attn_output, _ = self.cross_attention(
            prompt_encoding,  # クエリ: プロンプト
            response_encoding,  # キー: 回答
            response_encoding   # バリュー: 回答
        )
        
        # 残差接続（改善点）
        attn_output = attn_output + prompt_encoding
        
        # 結合して特徴表現を作成
        combined = torch.cat([attn_output, response_encoding], dim=2)
        combined = combined.squeeze(1)  # [batch_size, 1, input_dim*2] -> [batch_size, input_dim*2]
        
        # 特徴抽出
        response_features = self.feature_extractor(combined)
        
        return response_features
        
    def forward(self, prompt_emb, response_a_emb, response_b_emb):
        """
        順伝播関数
        
        Args:
            prompt_emb (Tensor): プロンプトの埋め込み [batch_size, input_dim] または [batch_size, 1, input_dim]
            response_a_emb (Tensor): 回答Aの埋め込み [batch_size, input_dim] または [batch_size, 1, input_dim]
            response_b_emb (Tensor): 回答Bの埋め込み [batch_size, input_dim] または [batch_size, 1, input_dim]
            
        Returns:
            tuple: (logits, score_a, score_b)のタプル
                logits: 3クラスの確率 [batch_size, 3]
                score_a: 回答Aのスコア [batch_size, 1]
                score_b: 回答Bのスコア [batch_size, 1]
        """
        # 次元を追加してTransformerに適した形に変換（シーケンス長1）
        if prompt_emb.dim() == 2:
            prompt_emb = prompt_emb.unsqueeze(1)  # [batch_size, 1, input_dim]
            response_a_emb = response_a_emb.unsqueeze(1)  # [batch_size, 1, input_dim]
            response_b_emb = response_b_emb.unsqueeze(1)  # [batch_size, 1, input_dim]
        
        # 回答AとBを独立に処理
        response_a_features = self.process_response(prompt_emb, response_a_emb)
        response_b_features = self.process_response(prompt_emb, response_b_emb)
        
        # 個別のスコア計算（補助的な予測として）
        score_a = self.response_scorer(response_a_features)
        score_b = self.response_scorer(response_b_features)
        
        # 差分特徴を計算（改善点）
        diff_features = response_a_features - response_b_features
        
        # 回答AとB、および差分の特徴を結合して比較
        comparison_features = torch.cat([response_a_features, response_b_features, diff_features], dim=1)
        comparison_output = self.comparison_layer(comparison_features)
        
        # 最終的な分類（3クラス）
        logits = self.classifier(comparison_output)
        
        return logits, score_a, score_b


In [8]:

class ResponseDataset(Dataset):
    """
    回答評価のためのデータセットクラス
    """
    def __init__(self, prompt_embeddings, response_a_embeddings, response_b_embeddings, labels=None):
        """
        初期化関数
        
        Args:
            prompt_embeddings (numpy.ndarray): プロンプトの埋め込み [N, embed_dim]
            response_a_embeddings (numpy.ndarray): 回答Aの埋め込み [N, embed_dim]
            response_b_embeddings (numpy.ndarray): 回答Bの埋め込み [N, embed_dim]
            labels (numpy.ndarray, optional): ラベル [N]
                                             0: Aが好まれる, 1: Bが好まれる, 2: 同等
        """
        self.prompt_embeddings = torch.tensor(prompt_embeddings, dtype=torch.float32)
        self.response_a_embeddings = torch.tensor(response_a_embeddings, dtype=torch.float32)
        self.response_b_embeddings = torch.tensor(response_b_embeddings, dtype=torch.float32)
        
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.long)
        else:
            self.labels = None
            
        self.has_labels = labels is not None
        
    def __len__(self):
        """データセットの長さを返す"""
        return len(self.prompt_embeddings)
    
    def __getitem__(self, idx):
        """インデックスに対応するデータを返す"""
        if self.has_labels:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx],
                self.labels[idx]
            )
        else:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx]
            )



In [21]:

def train_model(model, train_loader, val_loader, n_epochs=15, lr=0.0001, 
                weight_decay=0.01, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    モデルを訓練する関数
    
    Args:
        model: 訓練するモデル
        train_loader: 訓練データローダー
        val_loader: 検証データローダー
        n_epochs: エポック数
        lr: 学習率
        weight_decay: 重み減衰パラメータ
        device: 使用デバイス ('cuda' or 'cpu')
        
    Returns:
        dict: 訓練結果（ベストモデル、履歴など）
    """
    # モデルをデバイスに移動
    model = model.to(device)
    print(f"Using device: {device}")
    
    # 損失関数と最適化アルゴリズム
    
    # 重み付きCrossEntropyLoss
    criterion = nn.CrossEntropyLoss()
    aux_criterion = nn.MSELoss()  # 補助的なスコア予測用
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # 学習率スケジューラ
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    
    # 結果記録用の変数
    best_val_acc = 0.0
    best_model_state = None
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': []
    }
    
    # エポックごとの訓練ループ
    for epoch in range(n_epochs):
        # 現在の学習率を記録
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)
        
        print(f"\nEpoch {epoch+1}/{n_epochs}, LR: {current_lr:.6f}")
        
        # 訓練フェーズ
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_data in train_loader:
            prompt_emb, resp_a_emb, resp_b_emb, labels = [b.to(device) for b in batch_data]
            
            # 勾配をゼロに初期化
            optimizer.zero_grad()
            
            # 順伝播
            outputs, score_a, score_b = model(prompt_emb, resp_a_emb, resp_b_emb)
            
            # メイン損失（分類）
            main_loss = criterion(outputs, labels)
            
            # 補助的な損失（スコア予測）
            # ラベルに基づいて期待されるスコアの差を設定
            expected_scores = torch.zeros_like(score_a)
            expected_scores[labels == 0] = 1.0  # Aが好まれる
            expected_scores[labels == 1] = -1.0  # Bが好まれる
            # 同等の場合は0のまま
            
            aux_loss = aux_criterion(score_a - score_b, expected_scores)
            
            # 総合損失
            loss = main_loss + 0.3 * aux_loss  # 補助損失の重みを調整（0.5から0.3に減少）
            
            # 逆伝播と最適化
            loss.backward()
            optimizer.step()
            
            # 統計の更新
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # 訓練統計の計算
        train_loss /= len(train_loader)
        train_acc = train_correct / train_total
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        
        # 検証フェーズ
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for batch_data in val_loader:
                prompt_emb, resp_a_emb, resp_b_emb, labels = [b.to(device) for b in batch_data]
                
                # 順伝播
                outputs, _, _ = model(prompt_emb, resp_a_emb, resp_b_emb)
                
                # 損失の計算
                loss = criterion(outputs, labels)
                
                # 統計の更新
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                
                # 予測とターゲットを記録
                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())
        
        # 検証統計の計算
        val_loss /= len(val_loader)
        val_preds = np.array(val_preds)
        val_targets = np.array(val_targets)
        val_acc = accuracy_score(val_targets, val_preds)
        
        # 混同行列を計算
        conf_matrix = confusion_matrix(val_targets, val_preds)
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print("Confusion Matrix:")
        print(conf_matrix)
        
        # 学習率スケジューラを更新
        scheduler.step(val_acc)
        
        # ベストモデルの保存
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            print(f"New best model with validation accuracy: {val_acc:.4f}")
    
    # 訓練終了後、ベストモデルを復元
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        
    return {
        'model': model,
        'best_val_acc': best_val_acc,
        'history': history
    }


def evaluate_model(model, test_loader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    モデルをテストデータで評価する関数
    
    Args:
        model: 評価するモデル
        test_loader: テストデータローダー
        device: 使用デバイス
        
    Returns:
        dict: 評価結果（精度、混同行列など）
    """
    model.eval()
    test_preds = []
    test_targets = []
    
    with torch.no_grad():
        for batch_data in test_loader:
            prompt_emb, resp_a_emb, resp_b_emb, labels = [b.to(device) for b in batch_data]
            
            # 順伝播
            outputs, _, _ = model(prompt_emb, resp_a_emb, resp_b_emb)
            
            _, predicted = torch.max(outputs, 1)
            test_preds.extend(predicted.cpu().numpy())
            test_targets.extend(labels.cpu().numpy())
    
    # 評価指標の計算
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    
    accuracy = accuracy_score(test_targets, test_preds)
    conf_matrix = confusion_matrix(test_targets, test_preds)
    f1 = f1_score(test_targets, test_preds, average='weighted')
    
    # クラスごとの精度
    class_accuracies = []
    for i in range(3):
        class_mask = test_targets == i
        if np.sum(class_mask) > 0:
            class_acc = accuracy_score(test_targets[class_mask], test_preds[class_mask])
            class_accuracies.append(class_acc)
        else:
            class_accuracies.append(0.0)
    
    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'confusion_matrix': conf_matrix,
        'class_accuracies': class_accuracies,
        'predictions': test_preds,
        'targets': test_targets
    }

In [30]:
def train_multimodal_model(model, train_loader, val_loader, n_epochs=20, lr=0.001, 
                         device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    マルチモーダルモデルを訓練する関数
    
    Args:
        model: 訓練するモデル
        train_loader: 訓練データローダー
        val_loader: 検証データローダー
        n_epochs: エポック数
        lr: 学習率
        device: 使用デバイス ('cuda' or 'cpu')
        
    Returns:
        dict: 訓練結果（ベストモデル、履歴など）
    """
    # モデルをデバイスに移動
    model = model.to(device)
    print(f"Using device: {device}")
    
    # 損失関数と最適化アルゴリズム
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # 学習率スケジューラ
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    
    # 結果記録用の変数
    best_val_acc = 0.0
    best_model_state = None
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': []
    }
    
    # エポックごとの訓練ループ
    for epoch in range(n_epochs):
        # 現在の学習率を記録
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)
        
        print(f"\nEpoch {epoch+1}/{n_epochs}, LR: {current_lr:.6f}")
        
        # 訓練フェーズ
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_data in train_loader:
            # バッチデータを展開
            # この部分はDataLoaderの実装に合わせて修正する必要がある
            (prompt_emb, resp_a_emb, resp_b_emb, 
             ling_feat_a, ling_feat_b, 
             read_a, read_b, 
             rel_a, rel_b, 
             sem_a, sem_b, 
             labels) = [b.to(device) for b in batch_data]
            
            # 勾配をゼロに初期化
            optimizer.zero_grad()
            
            # 順伝播
            outputs = model(prompt_emb, resp_a_emb, resp_b_emb, 
                          ling_feat_a, ling_feat_b, 
                          read_a, read_b, 
                          rel_a, rel_b, 
                          sem_a, sem_b)
            
            # 損失計算
            loss = criterion(outputs, labels)
            
            # 逆伝播と最適化
            loss.backward()
            optimizer.step()
            
            # 統計の更新
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # 訓練統計の計算
        train_loss /= len(train_loader)
        train_acc = train_correct / train_total
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        
        # 検証フェーズ
        val_metrics = evaluate_multimodal_model(model, val_loader, criterion, device)
        val_loss, val_acc, val_preds, val_targets, conf_matrix = val_metrics
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print("Confusion Matrix:")
        print(conf_matrix)
        
        # 学習率スケジューラを更新
        scheduler.step(val_acc)
        
        # ベストモデルの保存
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            print(f"New best model with validation accuracy: {val_acc:.4f}")
    
    # 訓練終了後、ベストモデルを復元
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        
    return {
        'model': model,
        'best_val_acc': best_val_acc,
        'history': history
    }

def evaluate_multimodal_model(model, data_loader, criterion=None, device='cuda'):
    """
    マルチモーダルモデルを評価する関数
    
    Args:
        model: 評価するモデル
        data_loader: データローダー
        criterion: 損失関数（省略可）
        device: 使用デバイス
        
    Returns:
        tuple: (損失, 精度, 予測, ターゲット, 混同行列)
    """
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for batch_data in data_loader:
            # バッチデータを展開
            (prompt_emb, resp_a_emb, resp_b_emb, 
             ling_feat_a, ling_feat_b, 
             read_a, read_b, 
             rel_a, rel_b, 
             sem_a, sem_b, 
             labels) = [b.to(device) for b in batch_data]
            
            # 順伝播
            outputs = model(prompt_emb, resp_a_emb, resp_b_emb, 
                          ling_feat_a, ling_feat_b, 
                          read_a, read_b, 
                          rel_a, rel_b, 
                          sem_a, sem_b)
            
            # 損失計算（クライテリオンが提供されている場合）
            if criterion is not None:
                loss = criterion(outputs, labels)
                total_loss += loss.item()
            
            # 予測
            _, predicted = torch.max(outputs, 1)
            
            # 予測とターゲットを記録
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    
    # 統計の計算
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    accuracy = accuracy_score(all_targets, all_preds)
    conf_matrix = confusion_matrix(all_targets, all_preds)
    
    # 平均損失を計算（クライテリオンが提供されている場合）
    avg_loss = total_loss / len(data_loader) if criterion is not None else 0.0
    
    return avg_loss, accuracy, all_preds, all_targets, conf_matrix


class MultiModalDataset(Dataset):
    """
    マルチモーダル特徴を持つデータセットクラス
    """
    def __init__(self, prompt_embeddings, response_a_embeddings, response_b_embeddings, 
                 linguistic_features_a, linguistic_features_b,
                 readability_a, readability_b,
                 relation_a, relation_b,
                 semantic_a, semantic_b,
                 labels=None):
        """
        初期化関数
        
        Args:
            prompt_embeddings: プロンプトの埋め込み
            response_a_embeddings: 回答Aの埋め込み
            response_b_embeddings: 回答Bの埋め込み
            linguistic_features_a: 回答Aの言語的特徴
            linguistic_features_b: 回答Bの言語的特徴
            readability_a: 回答Aの可読性スコア
            readability_b: 回答Bの可読性スコア
            relation_a: プロンプトと回答Aの関係特徴
            relation_b: プロンプトと回答Bの関係特徴
            semantic_a: 回答Aの意味的特徴
            semantic_b: 回答Bの意味的特徴
            labels: ラベル（0: Aが好まれる, 1: Bが好まれる, 2: 同等）
        """
        self.prompt_embeddings = torch.tensor(prompt_embeddings, dtype=torch.float32)
        self.response_a_embeddings = torch.tensor(response_a_embeddings, dtype=torch.float32)
        self.response_b_embeddings = torch.tensor(response_b_embeddings, dtype=torch.float32)
        
        self.linguistic_features_a = torch.tensor(linguistic_features_a, dtype=torch.float32)
        self.linguistic_features_b = torch.tensor(linguistic_features_b, dtype=torch.float32)
        
        self.readability_a = torch.tensor(readability_a, dtype=torch.float32)
        self.readability_b = torch.tensor(readability_b, dtype=torch.float32)
        
        self.relation_a = torch.tensor(relation_a, dtype=torch.float32)
        self.relation_b = torch.tensor(relation_b, dtype=torch.float32)
        
        self.semantic_a = torch.tensor(semantic_a, dtype=torch.float32)
        self.semantic_b = torch.tensor(semantic_b, dtype=torch.float32)
        
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.long)
        else:
            self.labels = None
            
        self.has_labels = labels is not None
        
    def __len__(self):
        """データセットの長さを返す"""
        return len(self.prompt_embeddings)
    
    def __getitem__(self, idx):
        """インデックスに対応するデータを返す"""
        if self.has_labels:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx],
                self.linguistic_features_a[idx],
                self.linguistic_features_b[idx],
                self.readability_a[idx],
                self.readability_b[idx],
                self.relation_a[idx],
                self.relation_b[idx],
                self.semantic_a[idx],
                self.semantic_b[idx],
                self.labels[idx]
            )
        else:
            return (
                self.prompt_embeddings[idx],
                self.response_a_embeddings[idx],
                self.response_b_embeddings[idx],
                self.linguistic_features_a[idx],
                self.linguistic_features_b[idx],
                self.readability_a[idx],
                self.readability_b[idx],
                self.relation_a[idx],
                self.relation_b[idx],
                self.semantic_a[idx],
                self.semantic_b[idx]
            )

In [22]:
# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
prompt_embeddings = fine_tuned_embedding_p
response_a_embeddings = fine_tuned_embedding_A
response_b_embeddings = fine_tuned_embedding_B

num_samples = prompt_embeddings.shape[0]
embed_dim = prompt_embeddings.shape[1]

print(num_samples)
print(embed_dim)


# ラベルがGPU上にある場合
if isinstance(labels, torch.Tensor) and labels.is_cuda:
    labels_cpu = labels.cpu().numpy()
else:
    labels_cpu = labels

train_idx, test_idx = train_test_split(
    np.arange(num_samples), test_size=0.2, random_state=42, stratify=labels_cpu
)

train_idx, val_idx = train_test_split(
        train_idx, test_size=0.25, random_state=42, stratify=labels[train_idx]
)
    

# インデックスに基づいてデータを分割
p_train, r_a_train, r_b_train = prompt_embeddings[train_idx], response_a_embeddings[train_idx], response_b_embeddings[train_idx]
p_val, r_a_val, r_b_val = prompt_embeddings[val_idx], response_a_embeddings[val_idx], response_b_embeddings[val_idx]
p_test, r_a_test, r_b_test = prompt_embeddings[test_idx], response_a_embeddings[test_idx], response_b_embeddings[test_idx]

y_train, y_val, y_test = labels[train_idx], labels[val_idx], labels[test_idx]

# ========== データローダーの作成 ==========
# データセットの作成
train_dataset = ResponseDataset(p_train, r_a_train, r_b_train, y_train)
val_dataset = ResponseDataset(p_val, r_a_val, r_b_val, y_val)
test_dataset = ResponseDataset(p_test, r_a_test, r_b_test, y_test)

# データローダーの作成
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


# ========== モデルの作成と訓練 ==========
model = ImprovedResponseEvaluator(
    input_dim=384,      # 入力の次元数
    hidden_dim=256,     # 隠れ層の次元数
    num_heads=6,        # 注意ヘッドの数
    dropout=0.15        # ドロップアウト率
)


# モデルの要約を表示
print(model)

train_model(model, train_loader, val_loader, n_epochs=15)
eval_results = evaluate_model(model, test_loader)
print(f"Test Accuracy: {eval_results['accuracy']:.4f}")
print(f"Confusion Matrix:\n{eval_results['confusion_matrix']}")

57477
384
ImprovedResponseEvaluator(
  (cross_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
  )
  (feature_extractor): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): Dropout(p=0.15, inplace=False)
    (3): ReLU()
  )
  (comparison_layer): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): Dropout(p=0.15, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (6): Dropout(p=0.15, inplace=False)
    (7): ReLU()
  )
  (classifier): Linear(in_features=128, out_features=3, bias=True)
  (response_scorer): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
  )
)
Using device: cuda

Epoch 1/15, LR: 0.